<a href="https://colab.research.google.com/github/HSS107048212/hoop4s/blob/main/%E6%9C%88%E7%B5%90Tpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
pip install pygsheets

In [155]:
import os
import re
import smtplib
import pandas as pd
import gspread

from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Remaining code has been shortened for brevity...

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import numpy as np
import pygsheets

#HOME_PATH = '/home/iss/realplus/'
#HOME_PATH = '/home/iss/realplus/'
HOME_PATH = '/content/' # path of Colab for testing

# Authorization
gc = pygsheets.authorize(service_file=HOME_PATH+'orderdata-copyto-parttime-5c19ba8fcba4.json')

# Google sheet URL and authentication
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1eHm_1rIPSQgUqYNStKq8kfMnGiQnfUDhxGEj1I15Fls/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("10月份客戶交易明細表")

df = wks.get_as_df(start='A6', numerize=False )
# 新的列名列表
new_column_names = ['日期', '單別', '單據編號', '商品編號', 'Unnamed: 4', '商品名稱', 'Unnamed: 6',
                    'Unnamed: 7', 'Unnamed: 8', '數量', '單價', '金額']
# 更新 DataFrame 的列名
df.columns = new_column_names

# 將所有空字符串 ('') 替換為 NaN
df.replace('', np.nan, inplace=True)

df


/usr/local/lib/python3.10/dist-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


,日期,單別,單據編號,商品編號,Unnamed: 4,商品名稱,Unnamed: 6,Unnamed: 7,Unnamed: 8,數量,單價,金額
0,客戶編號:,000,NaN,客戶名稱:,NaN,NaN,NaN,NaN,NaN,統一編號:,NaN,NaN
1,2023/10/02,銷貨,202310020002,S00801-0,NaN,ST有善田園茂谷汽水,NaN,1.00,NaN,NaN,50.00,50
2,NaN,NaN,NaN,業務:,發票號碼:,TQ47966456,稅金:,0,NaN,NaN,NaN,50
3,2023/10/02,銷貨,202310020003,S00801-0,NaN,ST有善田園茂谷汽水,NaN,1.00,NaN,NaN,50.00,50
4,NaN,NaN,NaN,S00802-0,NaN,ST有善田園洛神黑麥汁,NaN,1.00,NaN,NaN,50.00,50
...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,NaN,業務:,發票號碼:,TQ47966539,稅金:,0,NaN,NaN,NaN,100
296,2023/10/31,銷貨,202310310006,S00802-0,NaN,ST有善田園洛神黑麥汁,NaN,1.00,NaN,NaN,50.00,50
297,NaN,NaN,NaN,業務:,發票號碼:,TQ47966540,稅金:,0,NaN,NaN,NaN,50
298,合計:,NaN,NaN,NaN,NaN,NaN,稅金:,0,NaN,NaN,NaN,150


In [156]:
# Define the column names and the value to search for based on the user's request
column_to_check = '日期'
value_to_search_for = '客戶編號:'
column_to_return = '單別'

# Find the rows where column_to_check equals value_to_search_for
matching_rows = df[df[column_to_check] == value_to_search_for]

# Get the values from column_to_return for these rows
matching_values = matching_rows[column_to_return].tolist()

# Return the matching row indices and their corresponding values in column_to_return
result_indices = matching_rows.index.tolist()
result = dict(zip(result_indices, matching_values))

import re
# Regular expression to match the pattern '1 uppercase letter followed by 6 digits'
pattern = r'^[A-Z]\d{6}$'

# Filtering the dictionary
result = {k: v for k, v in result.items() if re.match(pattern, v)}

print(result)
print(len(result))

{159: 'A862233', 165: 'B169595', 169: 'B231568', 173: 'D995235', 177: 'E789764', 181: 'H679207', 195: 'J033579', 199: 'J063816', 203: 'J200318', 218: 'J436617', 240: 'J692508', 244: 'J925577', 251: 'J933990', 260: 'M829221', 264: 'N221983', 276: 'N981930', 286: 'O951455', 292: 'R313833'}
18


In [157]:
index = min(list(result.keys()))  # starting index, adjust as necessary
a = []
# Loop until the end of the DataFrame
while index < len(df)-2:  # assuming df is your DataFrame and has at least 2 rows
    #print(index)
    # Check your conditions
    if df["Unnamed: 6"][index+2] == "稅金:" and df["Unnamed: 6"][index+1] == "稅金:" and pd.isnull(df["Unnamed: 6"][index]):
        amount = int(df['金額'][index+2].replace(',', ''))
        print(f"{index+2}:{amount}")  # Corrected the quotes inside the f-string
        a.append(amount)
    #else: print("NO")
    # Increment index at the end of each iteration
    index += 1
#print(a)

164:480
168:160
172:100
176:110
180:1200
194:1030
198:50
202:60
217:2298
239:7390
243:220
250:1220
259:1030
263:350
275:1950
285:819
291:160
298:150


In [158]:
# Checking if the lengths of result.values() and a are the same
len_result_values = len(result.values())
len_a = len(a)
lengths_match = len_result_values == len_a

# Creating the 'record' dictionary if lengths match
record = {}
if lengths_match:
    record = dict(zip(result.values(), a))

lengths_match

True

In [159]:
# 轉換為新的字典格式
transformed_record = {
    '社員編碼': list(record.keys()),
    '本月點數': list(record.values())
}

transformed_record


{'社員編碼': ['A862233',
  'B169595',
  'B231568',
  'D995235',
  'E789764',
  'H679207',
  'J033579',
  'J063816',
  'J200318',
  'J436617',
  'J692508',
  'J925577',
  'J933990',
  'M829221',
  'N221983',
  'N981930',
  'O951455',
  'R313833'],
 '本月點數': [480,
  160,
  100,
  110,
  1200,
  1030,
  50,
  60,
  2298,
  7390,
  220,
  1220,
  1030,
  350,
  1950,
  819,
  160,
  150]}

In [160]:
# Google sheet URL and authentication
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/19lfmsgopDkKvIw5TFqx1QptgVu4OOQz8EwC4L7SRQxk/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("修改")

df = wks.get_as_df()

# 將所有空字符串 ('') 替換為 0
df["本月點數"].replace('', 0, inplace=True)
df["累積點數"].replace('', 0, inplace=True)


In [161]:
tmp=0
for id in transformed_record['社員編碼']:
  if df.loc[df['社員編碼'] == id, '本月點數']
    record
  df.loc[df['社員編碼'] == id, '本月點數'].values[0] += transformed_record['本月點數'][tmp]
  df.loc[df['社員編碼'] == id, '累積點數'].values[0] += df.loc[df['社員編碼'] == id, '本月點數'].values[0]
  tmp+=1

SyntaxError: ignored

In [ ]:
df.loc[df['社員編碼'] == id, '本月點數']

In [ ]:
df.loc[df['社員編碼'] == id, '本月點數'].values[0]

In [ ]:
tmp=0
transformed_record['本月點數'][tmp]

In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os


def email_sent(Tosearch):
  # 設定寄件者的電子郵件地址和密碼作為環境變數
  os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
  os.environ["COOP_PASSWORD"] = "coop4s@nthu"
  filtered_data = Order_data[Order_data['訂單編號'] == Tosearch]
  Selected_filtered_data = filtered_data.iloc[:, [2, 3, 5, -2, -1]]
  print("===================================")
  print(Selected_filtered_data)
  print("===================================\n")
  try:
    AreYouSure = int(input("請確定，上述關於該訂單資訊是否正確，如正確請按1 \n"))
    print("===================================\n")
    if AreYouSure == 1 :
        # Your loop for sending emails
        for index, row in filtered_data.iterrows():
            recipient_name = row["訂購者姓名"]
            order_num = row["訂單編號"]
            product_quantities = row[6:-2]
            total_amount = row["小計"]

            # Create email content
            email_content = f"Dear {recipient_name},\n十里方圓合作社提醒您，您已付款購買的產品詳情如下，請確認:\n\n"
            email_content += f"===================================\n"
            email_content += f"系統自動生成的訂單編號: {order_num} \n"
            email_content += f"==================================="
            for product, quantity in product_quantities.items():
                if quantity != "" :
                  # Check if the product is in the "商品名稱" column
                  if product in Price_data["商品全稱"].values:
                      # If the product is found, you can retrieve its "銷售價格"
                      sale_price = Price_data.loc[Price_data["商品全稱"] == product, "銷售價格"].values[0]
                  email_content += f"{product}: {quantity} 件，該商品單價是{sale_price}元\n"
            email_content += f"===================================\n"
            email_content += f"付款金額小計: {total_amount} 新台幣\n"
            email_content += f"獲得T-point: {total_amount} 點數\n"


            # Email settings
            sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
            sender_password = "jwsscmoiffdyukmm"
            recipient_email = str(row["電子郵件地址"])  # 確保將電子郵件地址轉換為字符串

            subject = "【十里方圓合作社】完成結帳通知"
            message = MIMEMultipart()
            message["From"] = sender_email
            message["To"] = recipient_email
            message["Subject"] = subject
            message.attach(MIMEText(email_content, "plain"))

            try:
                # Connect to SMTP server
                server = smtplib.SMTP("smtp.gmail.com", 587)
                server.starttls()
                server.login(sender_email, sender_password)

                # Send the email
                server.sendmail(sender_email, recipient_email, message.as_string())

                # Close the SMTP server
                server.quit()

                print(f"Email sent to {recipient_name}, recipient's email address: {recipient_email}")
            except Exception as e:
                print(f"Failed to send email to {recipient_name}. Error: {str(e)}")
    else:
        print("停止輸入")
  except ValueError:
    print("停止輸入")
